In [1]:
import torch
import torchvision
import cv2
from subprocess import call

In [2]:
model = torchvision.models.resnet18(pretrained=True)

/home/pranjal-ubuntu/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pranjal-ubuntu/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
network = torch.load("/home/pranjal-ubuntu/Downloads/resnet_model.pt")

In [4]:
smoothed_angle = 0
xs = []
ys = []

In [5]:

img = cv2.imread('steering.jpg')

In [6]:
rows,cols,channels = img.shape

In [7]:
print(rows)
print(cols)

240
240


In [8]:
with open("data.txt") as f:
    for line in f:
        xs.append("driving_dataset" + line.split()[0])
        ys.append(float(line.split()[1]))

In [9]:
num_images = len(xs)
print(num_images)

45406


In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [13]:

for i in range(num_images):
    full_image = cv2.imread("driving_dataset/" + str(i) + ".jpg", cv2.IMREAD_COLOR)
    image = cv2.resize(full_image, (256, 256))
    cv2.imshow("image",image)
    image = torch.from_numpy(image)
    image = image.to(device)
    image = image.float()
    image = image.reshape(1, 3, 256, 256)
    predicted = network(image)
    smoothed_angle += 0.3 * pow(abs((predicted - smoothed_angle)), 2.0 / 3.0) * (predicted - smoothed_angle) / abs(predicted - smoothed_angle)
    smoothed_angle = smoothed_angle.item()
    print(f"Predicted steering angle : {smoothed_angle}")
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering", dst)
    cv2.waitKey(1)
    i = i+1


Predicted steering angle : -2.572908639907837
Predicted steering angle : -4.794498920440674
Predicted steering angle : -7.183904647827148
Predicted steering angle : -9.08810806274414
Predicted steering angle : -8.680208206176758
Predicted steering angle : -9.820165634155273
Predicted steering angle : -11.915932655334473
Predicted steering angle : -13.32911205291748
Predicted steering angle : -14.857433319091797
Predicted steering angle : -13.130354881286621
Predicted steering angle : -10.851922035217285
Predicted steering angle : -8.225091934204102
Predicted steering angle : -5.445172309875488
Predicted steering angle : -2.8767714500427246
Predicted steering angle : -0.4797816276550293
Predicted steering angle : 0.6796153783798218
Predicted steering angle : 1.4767332077026367
Predicted steering angle : 0.9005012512207031
Predicted steering angle : -0.3584331274032593
Predicted steering angle : -1.9803482294082642
Predicted steering angle : -2.3986990451812744
Predicted steering angle :

KeyboardInterrupt: 

In [14]:
cv2.destroyWindow("steering")
cv2.destroyWindow("image")